# Imports and setting up viz

NB : conda env1 on Mac, lam1env on spirit (Python3.12)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

#import personnal tools
import sys
sys.path.append('../../python_tools/')
from tools import *
from tools_mapping import *

In [ ]:
rivers = cfeature.NaturalEarthFeature('physical', 'rivers_lake_centerlines', '10m',edgecolor=(0, 0, 0, 0.3), facecolor='none')

# Load files

LAM output analysis.

Simu irr et simu no-irr.

## Area and period selection

In [ ]:
# lon_min=-13
# lon_max=6
# lat_min=32
# lat_max=49

date_min = '2010-01-01'
date_max = '2022-12-31'

## Sims

In [ ]:
# noirr_dir='../../../JZ_simu_outputs/LAM/LAM_1000_40/noirr_2010_2022'
# irr_dir='../../../JZ_simu_outputs/LAM/LAM_1000_40/irr_2010_2022'
noirr_dir='../../../JZ_simu_outputs/LAM/LAM_1500_60/noirr'
irr_dir='../../../JZ_simu_outputs/LAM/LAM_1500_60/irr'

In [ ]:
TS_flag=False

In [ ]:
#open netcdf files
if TS_flag:
    filename = '{}/*/SRF/TS_MO/*.nc'.format(noirr_dir)
elif not TS_flag:
    filename = '{}/*/SRF/MO/*sechiba_history.nc'.format(noirr_dir)

ORCnoirr0 = xr.open_mfdataset(filename)
ORCnoirr = ORCnoirr0.rename({'time_counter':'time'})
ORCnoirr.attrs['name'] = 'no_irr'
ORCnoirr.attrs['plot_color'] = "red"
ORCnoirr = ORCnoirr.sel(lon=slice(lon_min,lon_max),lat=slice(lat_min,lat_max))

ORCnoirr['snowmelt'] = ORCnoirr['snowmelt'] * 24 * 3600
ORCnoirr['snowmelt'].attrs['units'] = 'mm/day'

ORCnoirr['snow_contrib'] = ORCnoirr['snowmelt'] / (ORCnoirr['snowmelt'] + ORCnoirr['rain']) * 100
ORCnoirr['snow_contrib'].attrs['units'] = '%'

ORCnoirr

In [ ]:
if TS_flag:
    filename = '{}/*/SRF/TS_MO/*.nc'.format(noirr_dir)
elif not TS_flag:
    filename = '{}/*/SRF/MO/*history.nc'.format(irr_dir)

ORCirr0 = xr.open_mfdataset(filename)
ORCirr = ORCirr0.rename({'time_counter':'time'})
ORCirr.attrs['name'] = 'irr'
ORCirr.attrs['plot_color'] = "#0C5DA5"
ORCirr = ORCirr.sel(lon=slice(lon_min,lon_max),lat=slice(lat_min,lat_max))

ORCirr['snowmelt'] = ORCirr['snowmelt'] * 24 * 3600
ORCirr['snowmelt'].attrs['units'] = 'mm/day'

ORCirr['snow_contrib'] = ORCirr['snowmelt'] / (ORCirr['snowmelt'] + ORCirr['rain']) * 100
ORCirr['snow_contrib'].attrs['units'] = '%'

ORCirr

In [ ]:
if not TS_flag:
    # #manually define irrig_deficit as netirrig-irrigation in ORC file
    # ORCirr['irrig_deficit'] = ORCirr['netirrig'] - ORCirr['irrigation']
    # #make irrig_deficit units mm/day
    # ORCirr['irrig_deficit'].attrs['units'] = 'mm/day'
    # ORCirr['irrig_frac'] = ORCirr['irrigmap_dyn']/ORCirr['Areas']

    ORCnoirr['irrig_frac'] = ORCirr['irrigmap_dyn']/ORCirr['Areas'] * 100
    ORCnoirr['irrig_frac'].attrs['units'] = '%'
    ORCirr['irrig_frac'] = ORCirr['irrigmap_dyn']/ORCirr['Areas'] * 100
    ORCirr['irrig_frac'].attrs['units'] = '%'

## Obs

### Irrigation

In [ ]:
load_Ebro_obs=False

In [ ]:
if (load_Ebro_obs):
    filename = '../../../obs/SM_based_inversion_approach_EBRO_Irrigation.nc'
    obsEbro = xr.open_dataset(filename)
    obsEbro = obsEbro.rename({'Time':'time'})
    obsEbro = obsEbro.rename({'Longitude':'lon'})
    obsEbro = obsEbro.rename({'Latitude':'lat'})
    obsEbro = obsEbro.rename({'Irrigation':'irrigation'})
    #make lon and lat coordinates and not variables
    # obsEbro = obsEbro.set_coords(['lon','lat'])

    obsEbro = regrid_to_lon_lat(obsEbro)
    obsEbro.attrs['name'] = 'obs_Ebro'
    obsEbro.attrs['plot_color'] = "black"

    #convert to mm/day
    obsEbro['irrigation'] = obsEbro['irrigation']/30
    obsEbro['irrigation'].attrs['units'] = 'mm/d'
    obsEbro['irrigation'].attrs['long_name'] = 'Irrigation'
    
    #resample obsEbro to monthly values
    obsEbro = obsEbro.resample(time='1M').mean()
    
    obsEbro

### GLEAM v4

In [ ]:
filename = '../../../obs/GLEAMv4.1a/SM/SMs*'
gleamSMs = xr.open_mfdataset(filename)
gleamSMs.attrs['name'] = 'GLEAMv4.1a_SMs'
gleamSMs.attrs['plot_color'] = "green"
#restrict lon lat
gleamSMs = gleamSMs.sel(lon=slice(lon_min,lon_max),lat=slice(lat_max,lat_min))
#rename var SM
gleamSMs = gleamSMs.rename({'SMs':'SM'})
gleamSMs['SM'].attrs['units'] = 'm3/m3'

#obtain mrsos and humtot equivalent (mm)
gleamSMs['mrsos'] = gleamSMs['SM'] * 0.1 * 1000
gleamSMs['mrsos'].attrs['units'] = 'mm'
gleamSMs['humtot'] = gleamSMs['SM'] * 2 * 1000
gleamSMs['humtot'].attrs['units'] = 'mm'

gleamSMs

In [ ]:
filename = '../../../obs/GLEAMv4.1a/SM/SMrz*'
gleamSMrz = xr.open_mfdataset(filename)
gleamSMrz.attrs['name'] = 'GLEAMv4.1a_SMrz'
gleamSMrz.attrs['plot_color'] = "green"
#restrict lon lat
gleamSMrz = gleamSMrz.sel(lon=slice(lon_min,lon_max),lat=slice(lat_max,lat_min))
#rename var SM
gleamSMrz = gleamSMrz.rename({'SMrz':'SM'})
gleamSMrz['SM'].attrs['units'] = 'm3/m3'

#obtain mrsos and humtot equivalent (mm)
gleamSMrz['mrsos'] = gleamSMrz['SM'] * 0.1 * 1000
gleamSMrz['mrsos'].attrs['units'] = 'mm'
gleamSMrz['humtot'] = gleamSMrz['SM'] * 2 * 1000
gleamSMrz['humtot'].attrs['units'] = 'mm'

gleamSMrz

In [ ]:
filename = '../../../obs/GLEAMv4.1a/E/*'
gleamE = xr.open_mfdataset(filename)
gleamE.attrs['name'] = 'GLEAMv4.1a'
gleamE.attrs['plot_color'] = "green"
#restrict lon lat
gleamE = gleamE.sel(lon=slice(lon_min,lon_max),lat=slice(lat_max,lat_min))
#rename var SM
gleamE['evap']=gleamE['E'] / 31
gleamE['evap'].attrs['units'] = 'mm/d'
gleamE

### ERA5-Land

In [ ]:
load_EraLand=False

In [ ]:
if load_EraLand:
    filename = '../../../obs/ERA5-Land.nc'
    eraland = xr.open_dataset(filename)
    eraland.attrs['name'] = 'ERA5-Land'
    rename_dict = {'longitude':'lon',
                'latitude':'lat',
                'valid_time':'time',
                'skt':'temp_sol',
                'sro':'runoff',
                'ro':'mrro',
                'ssr':'SW_net',
                'str':'LW_net',
                'sshf':'fluxsens',
                'slhf':'fluxlat',
                'tp':'precip',
                'src':'fastr'
                }
    eraland = eraland.rename(rename_dict)

    eraland['mrsos'] = eraland['swvl1']*0.1*1000
    eraland['mrsos'].attrs['units'] = 'mm'
    #average variables swl1 to swvl4
    eraland['lvl1-4_ave'] = (eraland['swvl1'] * 7 + eraland['swvl2'] * 21 + eraland['swvl3'] * 72 + eraland['swvl4'] * 189 )  / 289
    eraland['humtot'] = eraland['lvl1-4_ave']*2*1000
    eraland['humtot'].attrs['units'] = 'mm'

    eraland['evap'] = eraland['fluxlat'] * 0.408 * 0.0864
    eraland['evap'].attrs['units'] = 'mm/d'

    eraland

### ERA5

In [ ]:
#open ERA5 file
# filename = '../../../obs/ERA5_eval_2010_2022.nc'
filename = '../../../obs/ERA5/ERA5_large_eval.nc'

era = xr.open_mfdataset(filename)
era.attrs['name'] = 'ERA5'
era.attrs['plot_color'] = "red"
#define or edit vars
#name ERA5 vars like in LMDZOR outputs
dict =  {
        'longitude':'lon',
        'latitude':'lat',
        'e':'evap',
        'ro':'totrunoff',
        'tp':'precip',
        'u10':'u10m',
        'v10':'v10m',
        'p71.162' : 'uq',
        'p72.162' : 'vq',
        'sshf' : 'fluxsens',
        'slhf' : 'fluxlat',
        'si10' : 'sfcWind',
        'ssrd' : 'SWdnSFC',
        'strd' : 'LWdnSFC',
        'ssr' : 'SWnetSFC',
        'str' : 'LWnetSFC',
        }
era = era.rename(dict)
era=era.sel(time=slice(date_min, date_max))
era = era.sel(lon=slice(lon_min,lon_max),lat=slice(lat_max, lat_min))

#make evap positive in era
era['evap'] = -era['evap'] * 1000
era['evap'].attrs['units'] = 'mm/d'
era['precip'] = era['precip']*1000
era['precip'].attrs['units'] = 'mm/d'

era


## Period selection

In [ ]:
#select only JJA
# ORCnoirr = ORCnoirr.sel(time=ORCnoirr['time.season']=='JJA')
# ORCirr = ORCirr.sel(time=ORCirr['time.season']=='JJA')

In [ ]:
if (load_Ebro_obs):
    #match the period of Ebro data
    date_min='2016-01-01'
    date_max='2020-07-31'

    ORCnoirr = ORCnoirr.sel(time=slice(date_min,date_max))
    ORCirr = ORCirr.sel(time=slice(date_min,date_max))
    obsEbro = obsEbro.sel(time=slice(date_min,date_max))

## Interpolation and masks

In [ ]:
#create a masks of data points where irrig_frac >5%
irr_mask = (ORCirr['irrigmap_dyn']/ORCirr['Areas'])>0.05
con_mask=ORCnoirr['Contfrac']>0.95
ip_mask=polygon_to_mask(ORCnoirr, iberian_peninsula)

In [ ]:
#iberian peninsula ds
ip_ORCnoirr=ORCnoirr.where(con_mask).where(ip_mask)
ip_ORCirr=ORCirr.where(con_mask).where(ip_mask)

In [ ]:
#mask on irrigated areas only
irr_ORCirr=ORCirr.where(irr_mask)
irr_ORCnoirr=ORCnoirr.where(irr_mask)

ip_irr_ORCirr=ip_ORCirr.where(irr_mask)
ip_irr_ORCnoirr=ip_ORCnoirr.where(irr_mask)

In [ ]:
if (load_Ebro_obs):
    ebro_mask = obsEbro['irrigation']>0
    ORCirr_iObsEbro = ORCirr.interp_like(obsEbro)
    ebro_ORCirr_iObsEbro = ORCirr_iObsEbro.where(ebro_mask)
    obsEbro_iORC = obsEbro.interp_like(ORCirr)
    ebro_mask_iORCirr = obsEbro_iORC['irrigation']>0
    ebro_ORCirr = ORCirr.where(ebro_mask_iORCirr)

In [ ]:
#interpolate gleam to sim grid
gleamSMs_iORC = gleamSMs.interp_like(ORCnoirr)
gleamSMrz_iORC = gleamSMrz.interp_like(ORCnoirr)
gleamE_iORC = gleamE.interp_like(ORCnoirr)
#mask on Iberian Peninsula
ip_gleamSMs_iORC = gleamSMs_iORC.where(ip_mask)
ip_gleamSMrz_iORC = gleamSMrz_iORC.where(ip_mask)
ip_gleamE_iORC = gleamE_iORC.where(ip_mask)
#mask on irrigated areas only
ip_irr_gleamSMs_iORC = ip_gleamSMs_iORC.where(irr_mask)
ip_irr_gleamSMrz_iORC = ip_gleamSMrz_iORC.where(irr_mask)
ip_irr_gleamE_iORC = ip_gleamE_iORC.where(irr_mask)

In [ ]:
if load_EraLand:
    #interpolate eraland to sim grid
    eraland_iORC = eraland.interp_like(ORCnoirr)
    #mask on Iberian Peninsula
    ip_eraland_iORC = eraland_iORC.where(ip_mask)

In [ ]:
#interpolate era to sim grid and apply ip and irr masks
era_iORC = era.interp_like(ORCnoirr)
#mask on Iberian Peninsula
ip_era_iORC = era_iORC.where(ip_mask)
#mask on irrigated areas only
ip_irr_era_iORC = ip_era_iORC.where(irr_mask)

In [ ]:
if not (load_Ebro_obs):
    ebro_mask = polygon_to_mask(ORCnoirr, ebro)
    ebro_ORCirr = ip_ORCirr.where(ebro_mask)
    ebro_ORCnoirr = ip_ORCnoirr.where(ebro_mask)
    ebro_gleamE_iORC = ip_gleamE_iORC.where(ebro_mask)
    ebro_era_iORC = ip_era_iORC.where(ebro_mask)
    ebro_irr_ORCirr = ebro_ORCirr.where(irr_mask)
    ebro_irr_ORCnoirr = ebro_ORCnoirr.where(irr_mask)
    ebro_irr_gleamE_iORC = ebro_gleamE_iORC.where(irr_mask)
    ebro_irr_era_iORC = ebro_era_iORC.where(irr_mask)

## Diff datasets

In [ ]:
#irr - noirr
sim_diff = ORCirr - ORCnoirr
sim_diff.attrs['name'] = 'irr - noirr'
ip_sim_diff = ip_ORCirr - ip_ORCnoirr
ip_sim_diff.attrs['name'] = 'irr - noirr'

#mean
sim_diff_mean = sim_diff.mean(dim='time')
sim_diff_mean.attrs['name'] = 'irr - noirr mean'
ip_sim_diff_mean = ip_sim_diff.mean(dim='time')
ip_sim_diff_mean.attrs['name'] = 'irr - noirr mean'

# Maps

In [ ]:
ip_ORCirr['aei_sw_100'] = ip_ORCirr['aei_sw'] * 100
ip_ORCirr['aei_sw_100'].attrs['units'] = '%'

In [ ]:
ds=ip_ORCnoirr
# ds=ebro_ORCnoirr
var='snow_contrib' 

vmin = 0
vmax = 35
# vmin, vmax = None, None
# vmax=None

title=None
# title='off'

# clabel="Share of surface withdrawals (%)"
clabel=None

cmap=myvir

map_ave(ds, var, vmin=vmin, vmax=vmax, title=title, clabel=clabel, cmap=cmap, poly=None)

In [ ]:
#diff
ds1=ebro_ORCirr
ds2=obsEbro_iORC
var='irrigation'
vmax= 0.5
vmin=-0.5
# vmin, vmax=None, None
title='(a) Irrigation bias (2016-2020)'
# title=None
title='off'
cmap=emb_neutral
ylabel="Irrigation bias (mm/d)"
map_diff_ave(ds1, ds2, var,cmap=cmap, title=title, clabel=ylabel, vmin=vmin, vmax=vmax)

In [ ]:
#diff with significance
ds1=ORCirr   
ds2=ORCnoirr
var='evap'
vmax= 40
vmin=-40
vmin, vmax=None, None
title='Sensible heat flux difference (irr-no_irr, JJA, W/m²)'
title=None
map_diff_ave(ds1, ds2, var, title=title, vmin=vmin, vmax=vmax, sig=True)

In [ ]:
#relative difference
ds1=ip_ORCirr
ds2=ip_ORCnoirr
var='slowr'
vmax= 100
vmin=-100
title='off'
clabel='Slow reservoir difference (%)'
map_rel_diff_ave(ds1, ds2, var,cmap=emb_neutral, title=title, clabel=clabel, vmin=vmin, vmax=vmax)

In [ ]:
#Display map of var for ds on restricted area
var='soiltile'
ds = ebro_ORCirr
latmin=39.75
latmax=44.1
lonmin= -4.5
lonmax= 2.5
# latmin=41.
# latmax=42.5
# lonmin= 0.
# lonmax= 1.5

#restrict ds to latmin, latmax, lonmin, lonmax
ds = ds.where(ds.lat >= latmin, drop=True).where(ds.lat <= latmax, drop=True)
ds = ds.where(ds.lon >= lonmin, drop=True).where(ds.lon <= lonmax, drop=True)
plotvar = ds[var].mean(dim='time')
map_plotvar(plotvar, cmap=wet, vmin=0, vmax=0.8, title='Applied irrigation (mm/day)')

In [ ]:
#Display diff of var for ds on restricted area
var='irrigation'
ds1 = ORCirr
ds2 = obsEbro_iORC
latmin=39.75
latmax=44.1
lonmin= -4.5
lonmax= 2.5
# latmin=41.
# latmax=42.5
# lonmin= 0.
# lonmax= 1.5

#restrict ds to latmin, latmax, lonmin, lonmax
ds1 =ds1.where(ds1.lat >= latmin, drop=True).where(ds1.lat <= latmax, drop=True)
ds1 =ds1.where(ds1.lon >= lonmin, drop=True).where(ds1.lon <= lonmax, drop=True)
ds2 =ds2.where(ds2.lat >= latmin, drop=True).where(ds2.lat <= latmax, drop=True)
ds2 =ds2.where(ds2.lon >= lonmin, drop=True).where(ds2.lon <= lonmax, drop=True)

vmin=-0.5
vmax= 0.5
# vmin, vmax = None, None
map_diff_ave(ds1, ds2, var, vmin=vmin, vmax=vmax)

In [ ]:
#display seasonal maps of var
ds=ip_ORCirr
var='drainage'
vmin = 0
vmax = 6
# vmin, vmax = None, None
# vmax=None

title='Irrigation (for grid cells with irrig_frac >0.1%, +IP mask, mm/d)'
title='{} ({})'.format(ds[var].attrs['long_name'], ds[var].attrs['units'])

map_seasons(ds[var], vmin=vmin, vmax=vmax, title=title, cmap=wet)

In [ ]:
#Display map of 3D var over one dimension
var='soiltile'
ds = ORCirr
dim_nb=0
title=var + str(dim_nb+1)
plotvar = ds[var][:,dim_nb,:,:].mean(dim='time')
map_plotvar(plotvar, cmap=reds, vmin=0, vmax=1, title=title)

# Time series

In [ ]:
var='snowf'

# ds1=ebro_ORCirr
# ds2=obsEbro_iORC
# ds1 = ebro_irr_ORCnoirr
# ds2 = ebro_irr_ORCirr
# ds3 = ebro_irr_gleamE_iORC
# ds4 = ebro_irr_era_iORC
# ds1=ip_cci_iORC
ds1=ip_ORCnoirr
ds2=ip_ORCirr
ds3=ebro_ORCnoirr

# ds_list = [ds1, ds2, ds3, ds4]
ds_list=[ds1]#, ds2]

year_min = 2010
year_max = 2019
# title='Ebro irrigated areas average evaporation (mm/day)'
# title='Average irrigation requirement (irrigated grid cells only, mm/day)'
# title=var + ' (irrigated grid cells only)'
# title='off'
title=None

# ylabel="Irrigation (mm/d)"
time_series_ave(ds_list, var, ds_colors=True, year_min=year_min, year_max=year_max, title=title)
seasonal_cycle_ave(ds_list, var, ds_colors=True, year_min=year_min, year_max=year_max, title=title)
plt.grid()

In [ ]:
ds=ip_cci_iORC
ds_list=[ds]
var='nobs'

# lon=-3
# lat=41
# time_series_lonlat(ds_list, var, lon, lat)

time_series_ave(ds_list, var)

# Scatter plots

In [ ]:
var1='irrig_frac'
ds1=ip_ORCirr
# ds1=ds1.where(mask)

var2='netirrig'
ds2=ds1
# ds2=ds2.where(mask)

scatter_annual_mean(ds1, ds2, var1, var2, reg=True)

In [ ]:
var1='irrig_frac'
ds1=ip_ORCirr
# mask=irr_mask
# ds1=ds1.where(mask)

var2='netirrig'
ds2=ip_ORCirr
# ds2=ds2.where(mask)

seasons=['DJF','MAM', 'JJA', 'SON']
seasons=['SON']

scatter_vars_seasons(ds1, ds2, var1, var2, reg=True, coloring=True, seasons_list=seasons)

## With spatial aggregation

In [ ]:
var1='irrigation'
ds1=ip_ORCirr.mean(dim='lon').mean(dim='lat')
# mask=irr_mask
# ds1=ds1.where(mask)

var2='evap'
ds2=ip_sim_diff.mean(dim='lon').mean(dim='lat')
# ds2=ds2.where(mask)

scatter_vars(ds1, ds2, var1, var2, reg=True)

In [ ]:
var1='irrigation'
ds1=ip_ORCnoirr.mean(dim='lon').mean(dim='lat')
# mask=irr_mask
# ds1=ds1.where(mask)

var2='evap'
ds2=ip_sim_diff.mean(dim='lon').mean(dim='lat')
# ds2=ds2.where(mask)
months=[1,7,10]
scatter_vars_months(ds1, ds2, var1, var2, reg=False, coloring=True)